In [1]:
from ultralytics import YOLO


model = YOLO("yolov8n-military/weights/best.pt")

In [2]:
# Faire l'inférence sur une image
results = model("data/sample_game.png")  # Remplace par le chemin de ton image

# Afficher les résultats
results[0].show()  # Affiche l'image avec les boîtes de détection


image 1/1 c:\Users\tcastillo\Documents\ArgusModel\data\sample_game.png: 320x640 1 plane, 33.1ms
Speed: 4.2ms preprocess, 33.1ms inference, 111.4ms postprocess per image at shape (1, 3, 320, 640)


## ONXX Model

In [ ]:
"""
model.export(
    format='onnx',
    imgsz=640,  # ou 416 pour plus de performance
    simplify=True,
    opset=15,  # Sentis supporte jusqu'à opset 15
    batch=1,  # Batch size fixe de 1
    dynamic=False  # Pas de dimensions dynamiques
)
"""


import cv2
import numpy as np
import onnxruntime as ort

def preprocess_image(image_path, input_size=(640, 640)):
    # Charger l'image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image non trouvée ou format non supporté")

    # Redimensionner et normaliser
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image_rgb, input_size)
    input_image = resized / 255.0  # Normalisation [0,1]
    input_image = np.transpose(input_image, (2, 0, 1))  # HWC -> CHW
    input_image = np.expand_dims(input_image, axis=0)  # Ajouter batch dimension

    return input_image, image

def postprocess(outputs, img_shape, conf_thres=0.5):
    # Récupérer les détections
    detections = outputs[0]
    boxes = []
    confidences = []
    classes = ["Plane"]

    # Filtrer les détections selon le seuil de confiance
    for detection in detections:
        scores = detection[4:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_thres:
            # Récupérer les coordonnées de la boîte
            xc, yc, w, h = detection[:4]
            x1 = int((xc - w/2) * img_shape[1])
            y1 = int((yc - h/2) * img_shape[0])
            x2 = int((xc + w/2) * img_shape[1])
            y2 = int((yc + h/2) * img_shape[0])
            boxes.append([x1, y1, x2, y2])
            confidences.append(float(confidence))
            classes.append(int(class_id))

    return boxes, confidences, classes

def draw_boxes(image, boxes, confidences, classes):
    for box, conf, cls in zip(boxes, confidences, classes):
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"Class {cls}: {conf:.2f}"
        cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

# Chemin vers ton modèle ONNX et ton image
onnx_model_path = "yolov8n-military/weights/best.onnx"
image_path = "data/sample_game.png"  # Remplace par le chemin de ton image

# Charger le modèle ONNX
sess = ort.InferenceSession(onnx_model_path)

# Prétraiter l'image
input_image, original_image = preprocess_image(image_path)

# Faire l'inférence
outputs = sess.run(None, {"images": input_image.numpy()})

# Post-traiter les résultats
boxes, confidences, classes = postprocess(outputs, original_image.shape[:2])

# Dessiner les boîtes sur l'image
result_image = draw_boxes(original_image.copy(), boxes, confidences, classes)

# Afficher le résultat
cv2.imshow("Résultat", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Sauvegarder le résultat
cv2.imwrite("resultat.jpg", result_image)


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'